In [1]:
!pip install datasets transformers sentencepiece --quiet

     |████████████████████████████████| 451 kB 7.1 MB/s 
     |████████████████████████████████| 5.5 MB 52.9 MB/s 
     |████████████████████████████████| 1.3 MB 44.0 MB/s 
     |████████████████████████████████| 182 kB 62.4 MB/s 
     |████████████████████████████████| 115 kB 65.9 MB/s 
     |████████████████████████████████| 212 kB 62.7 MB/s 
     |████████████████████████████████| 127 kB 69.2 MB/s 
     |████████████████████████████████| 7.6 MB 51.7 MB/s 


In [6]:
import os 
import torch 
import numpy as np
import pandas as pd

from pathlib import Path 
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from transformers import TrainingArguments, Trainer, DataCollator
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split

In [3]:
import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
# df = pd.read_csv('/content/data (2).csv')

In [ ]:
# df.head()

In [ ]:
# df['Title'].isnull().sum()

In [ ]:
# rows_with_nan = []
# for index, row in df.iterrows():
#     is_nan_series = row.isnull()
#     if is_nan_series.any():
#         rows_with_nan.append(index)

In [ ]:
# df.info()

In [ ]:
# df.drop(rows_with_nan, inplace=True)

In [ ]:
# df.info()

In [ ]:
# df.to_csv('dataset.csv')

In [44]:
# df = pd.read_csv('/content/drive/MyDrive/nlp/dataset.csv')

In [45]:
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5239 entries, 0 to 5238
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5239 non-null   int64 
 1   keywords    5239 non-null   object
 2   Title       5239 non-null   object
 3   Article     5239 non-null   object
dtypes: int64(1), object(3)
memory usage: 163.8+ KB


In [46]:
# df.drop('Unnamed: 0', axis = 1, inplace = True)

In [47]:
# df=df.drop_duplicates(subset = ['Title', 'Article'], keep='first')

In [48]:
# df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4742 entries, 0 to 5238
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keywords  4742 non-null   object
 1   Title     4742 non-null   object
 2   Article   4742 non-null   object
dtypes: object(3)
memory usage: 148.2+ KB


In [49]:
# train, test = train_test_split(df, test_size=0.025)

In [50]:
# train.to_csv('/content/drive/MyDrive/nlp/train.csv')
# test.to_csv('/content/drive/MyDrive/nlp/test.csv')

In [53]:
dataset = load_dataset('csv', data_files={'train': '/content/drive/MyDrive/nlp/train.csv',
                                              'test': '/content/drive/MyDrive/nlp/test.csv'})
dataset = dataset.remove_columns(["Unnamed: 0","Article"])

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ce5223cc6e69ea05/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [54]:
dataset

DatasetDict({
    train: Dataset({
        features: ['keywords', 'Title'],
        num_rows: 4623
    })
    test: Dataset({
        features: ['keywords', 'Title'],
        num_rows: 119
    })
})

In [55]:
from transformers import AutoModelWithLMHead, AutoTokenizer, set_seed
set_seed(42)
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-common_gen")
tokenizer.pad_token = tokenizer.eos_token

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [56]:
padding = True
def tokenize_function(examples):
    return tokenizer(examples["keywords"], max_length=64,
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt")


tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("input_ids", "input_id")
tokenized_dataset = tokenized_dataset.rename_column("attention_mask", "attention_")
tokenized_dataset

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['keywords', 'Title', 'input_id', 'attention_'],
        num_rows: 4623
    })
    test: Dataset({
        features: ['keywords', 'Title', 'input_id', 'attention_'],
        num_rows: 119
    })
})

In [57]:
def tokenize_function(examples):
    return tokenizer(examples["Title"], max_length=128,
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt")
    
tokenized_datasets = tokenized_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("input_ids", "decoder_input_ids")
tokenized_datasets = tokenized_datasets.rename_column("attention_mask", "decoder_attention_mask")
tokenized_datasets = tokenized_datasets.rename_column("input_id", "input_ids")
tokenized_datasets = tokenized_datasets.rename_column("attention_", "attention_mask")
tokenized_datasets = tokenized_datasets.remove_columns(["keywords","Title"])
columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
tokenized_datasets.set_format(type='torch', columns=columns)

tokenized_datasets

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 4623
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 119
    })
})

In [58]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [59]:
training_args.num_train_epochs = 15
training_args.per_device_train_batch_size=1

In [60]:
tokenized_datasets['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
    num_rows: 4623
})

In [61]:
torch.cuda.empty_cache()

In [62]:
from torch.utils.data import DataLoader
t_datasets = tokenized_datasets.flatten()
train_dataloader = DataLoader(t_datasets['train'], shuffle=True, batch_size=2)
eval_dataloader = DataLoader(t_datasets['test'], batch_size=2)

In [63]:
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-common_gen")
model.config.pad_token_id = model.config.eos_token_id

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1136: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [64]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

In [65]:
from transformers import get_scheduler

num_epochs = 15
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [66]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [67]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [68]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        y = batch["decoder_input_ids"].to(device, dtype=torch.long)
        lm_labels = y[:, :].clone()
        lm_labels[y[:, :] == tokenizer.pad_token_id] = -100
        ids = batch["input_ids"].to(device, dtype=torch.long)
        mask = batch["attention_mask"].to(device, dtype=torch.long)
        outputs = model(input_ids=ids,
                        attention_mask=mask,
                        labels=lm_labels)
        
        loss = outputs[0]
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/34680 [00:00<?, ?it/s]

In [69]:
sentences = ["rock host netflix", "best t20 innings", "destination hip hop"]

inputs = tokenizer([sentence for sentence in sentences], return_tensors="pt", padding=True)

output_sequences = model.generate(
    input_ids=inputs["input_ids"].to(device, dtype=torch.long),
    attention_mask=inputs["attention_mask"].to(device, dtype=torch.long),
    do_sample=False,  # disable sampling to test if batching affects output
)

print(tokenizer.batch_decode(output_sequences, skip_special_tokens=True))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


['Rock to host Netflix’s first live broadcast in four years, starting with $5 million CAD', 'Best T20 innings of the year so far: Manisha Virbata and Man', 'Destination Hip-Hop Meets New New York City Millennials - a']


In [70]:
modelsavename= "/content/title_generation.pt"  # change the name here to save in a specific file (and restore below)
with open(modelsavename, "wb") as f:
    torch.save(model.state_dict(), f)

In [72]:
from transformers import AutoConfig, AutoModelWithLMHead, AutoTokenizer
MODEL_PATH = '/content/drive/MyDrive/nlp/title_generation_2.pt'
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-common_gen")
tokenizer.pad_token = tokenizer.eos_token
config = AutoConfig.from_pretrained("mrm8488/t5-base-finetuned-common_gen")
model = AutoModelWithLMHead.from_pretrained(MODEL_PATH, config = config)
model.to(device)
model.tie_weights()
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [79]:
sentences = ["Rock host netflix", "best t20 innings", "destination hip hop"]
MAX_LEN = 20
inputs = tokenizer([sentence for sentence in sentences], return_tensors="pt", padding=True)

output_sequences = model.generate(
    input_ids=inputs["input_ids"].to(device, dtype=torch.long),
    attention_mask=inputs["attention_mask"].to(device, dtype=torch.long),
    do_sample=True,
    max_length = MAX_LEN,
)

print(tokenizer.batch_decode(output_sequences, skip_special_tokens=True))

['Nicky Lee Hales: Will Rock host Netflix? Heard of him today on HG', 'Nadine Kerr gets best of his T20 innings with a solo century in India', 'YG returns to Aspen as a premium destination for hip hop fans: Yak']
